In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from matplotlib.pyplot import figure


df = pd.read_csv(f"../data/input_data/MAIN_DATASET.csv", index_col=0)
df.head()

##### Adding frequency to datetime to be able to decompose:

In [ ]:
index = pd.date_range(start='2016-01-03-12', periods=len(df), freq="H")


freq_series = pd.DataFrame(df["NO2_price"])
freq_series["Datetime"] = index
freq_series = freq_series.set_index(["Datetime"])
freq_series.head()

#### Decomposition of characteristics:

In [ ]:
result = seasonal_decompose(freq_series, model='add')
observedarray = result.observed
seasonalarray = result.seasonal
trendarray = result.trend
residarray = result.resid
observedarray.plot(figsize=(12,5), label = "Observed")
trendarray.plot(figsize=(12,5), label = "Trend")
residarray.plot(figsize=(12,5), label= "Residuals")
seasonalarray.plot(figsize=(12,5),label= "Seasonal", alpha = 0.4)
plt.ylabel("Decomposed Characteristics")
plt.legend()
plt.savefig("../figures/decomposed_characteristics.pdf", bbox_inches='tight')

In [ ]:
df_NVE = pd.read_csv(f"../data/input_data/magasin_statistikk_nve.csv", index_col=0)
df_NVE = df_NVE[df_NVE["omrnr" ]== 2]
df_NVE = df_NVE[df_NVE["omrType" ]== "EL"]
df_NVE['year'] = pd.DatetimeIndex(df_NVE['dato_Id']).year
df_NVE = df_NVE[df_NVE["year" ]>= 2016]

df_NVE.head()
df_NVE = df_NVE[["fyllingsgrad","dato_Id"]]
df_NVE = df_NVE.set_index(["dato_Id"])
df_NVE=df_NVE.sort_index()



index = pd.date_range(start='2016-01-03', periods=len(df_NVE), freq="W")

freq_series = pd.DataFrame(df_NVE["fyllingsgrad"])
freq_series["Datetime"] = index
freq_series = freq_series.set_index(["Datetime"])
freq_series.head()
result = seasonal_decompose(freq_series, model='add')
observedarray = result.observed
seasonalarray = result.seasonal
trendarray = result.trend
residarray = result.resid
observedarray.plot(figsize=(12,5), label = "Observed")
trendarray.plot(figsize=(12,5), label = "Trend")
residarray.plot(figsize=(12,5), label= "Residuals")
seasonalarray.plot(figsize=(12,5),label= "Seasonal", alpha = 0.4)
plt.title("Fyllingsgrad - Decomposed")
plt.ylabel("Decomposed Characteristics")

plt.legend()
plt.savefig("../figures/NVE_decomposed_characteristics.pdf", bbox_inches='tight')